# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [64]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from functools import reduce

df = pd.read_csv('data/titanic_train.csv')
train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'], axis = 1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df_group_Mean = df.groupby(['Pclass'])['Fare'].mean().reset_index()
df_group_Mode = df.groupby(['Pclass'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
df_group_Median = df.groupby(['Pclass'])['Fare'].median().reset_index()
df_group_Max = df.groupby(['Pclass'])['Fare'].max().reset_index()
df_group = [df_group_Mean, df_group_Mode, df_group_Median, df_group_Max]
temp = reduce(lambda left, right: pd.merge(left, right, on='Pclass'), df_group)
temp.columns = ['Pclass', 'Pclass_Fare_Mean', 'Pclass_Fare_Mode','Pclass_Fare_Median','Pclass_Fare_Max']
temp

,Pclass,Pclass_Fare_Mean,Pclass_Fare_Mode,Pclass_Fare_Median,Pclass_Fare_Max
0,1,84.154687,26.55,60.2875,512.3292
1,2,20.662183,13.00,14.2500,73.5000
2,3,13.675550,8.05,8.0500,69.5500


In [55]:
df = pd.merge(df, temp, on='Pclass', how = 'left')

In [56]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(len(num_features))
print(num_features)
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features].fillna(-1)
Scaler = MinMaxScaler()
df.head()

9
['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_Fare_Mean', 'Pclass_Fare_Mode', 'Pclass_Fare_Median', 'Pclass_Fare_Max']


,Pclass,Age,SibSp,Parch,Fare,Pclass_Fare_Mean,Pclass_Fare_Mode,Pclass_Fare_Median,Pclass_Fare_Max
0,3,22.0,1,0,7.2500,13.675550,8.05,8.0500,69.5500
1,1,38.0,1,0,71.2833,84.154687,26.55,60.2875,512.3292
2,3,26.0,0,0,7.9250,13.675550,8.05,8.0500,69.5500
3,1,35.0,1,0,53.1000,84.154687,26.55,60.2875,512.3292
4,3,35.0,0,0,8.0500,13.675550,8.05,8.0500,69.5500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [61]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_minus = df.drop(['Pclass_Fare_Mean', 'Pclass_Fare_Mode','Pclass_Fare_Median','Pclass_Fare_Max'], axis =1)
df_minus.head()
train_x = Scaler.fit_transform(df_minus)
Lo = LogisticRegression()
cross_val_score(Lo, train_x, train_Y, cv=5).mean()

0.7038635542329971

In [62]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_x = Scaler.fit_transform(df)
cross_val_score(Lo, train_x, train_Y, cv=5).mean()

0.7049872206659581

* 答題2: 少許的進步

In [65]:
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [71]:
df_Sex_age_Mean = df.groupby(['Sex'])['Age'].mean().reset_index()
df_Sex_age_Mode = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
df_Sex_age_Median = df.groupby(['Sex'])['Age'].median().reset_index()
df_Sex_age_Max = df.groupby(['Sex'])['Age'].max().reset_index()
temp2 = [df_Sex_age_Mean, df_Sex_age_Mode, df_Sex_age_Median, df_Sex_age_Max]
temp2 = reduce(lambda left, right: pd.merge(left, right, on ='Sex'), temp2)
temp2.columns =['Sex','Sex_age_Mean', 'Sex_age_Mode','Sex_age_Median','Sex_age_Max']
temp2


,Sex,Sex_age_Mean,Sex_age_Mode,Sex_age_Median,Sex_age_Max
0,female,27.915709,24.0,27.0,63.0
1,male,30.726645,19.0,29.0,80.0


In [80]:
#新特徵第二發+LOGISTIC REGRESSION
df2 = pd.merge(df, temp2, on='Sex', how='left')
num2_features = []
for i in df2.columns:
    if df2[i].dtype in ['float64', 'int64']:
        num2_features.append(i)
df2 =df2[num2_features].fillna(-1)

In [81]:
train_x = Scaler.fit_transform(df2)
cross_val_score(Lo, train_x, train_Y, cv =5).mean()

0.7834073637476873

In [83]:
#如果將新特徵1, 2 都加入去
df3 = pd.merge(df2, temp, on='Pclass', how = 'left')
train_x = Scaler.fit_transform(df3)
cross_val_score(Lo, train_x, train_Y, cv =5).mean()

0.7822774202448624

In [ ]:
#所以男女歲數 比起經濟能力 更能預測結果